 <div align="center">
  <h2>Extracción de los datos y elección de las 10 criptomondes</h2>
</div>

Importar las liberías necesarias

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import requests
from datetime import datetime
from pycoingecko import CoinGeckoAPI

crear una instancia de la clase CoinGeckoAPI

In [3]:
cg = CoinGeckoAPI()

La elección de las criptomonedas la estoy realizando por ranking de capitalización de mercado y para esto solo voy a pedir que se muestren las 30 monedas con mayor capitalización y la moneda con la que voy a comparar va ser el dolar

In [4]:
parametros = {
    "vs_currency": "usd", 
    "order": "market_cap_desc",
    "per_page": 30, 
    "page": 1,
}

data_coins_markets = cg.get_coins_markets(**parametros)
df_coins_markets = pd.DataFrame(data_coins_markets)

Con **df_coins_markets.head(10)** ya puedo ver las 10 criptomonedas con mayor capitalización del momento y ya tendría una elección previa, pero realizaré una análisis más profundo para tener una selección final

In [5]:
df_coins_markets.head(10)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,29346.000000,571046620112,1,6.163090e+11,9516498849,29653.000000,...,2.100000e+07,2.100000e+07,69045.000000,-57.49417,2021-11-10T14:24:11.849Z,67.810000,43180.46286,2013-07-06T00:00:00.000Z,None,2023-08-15T05:41:23.397Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1840.460000,221132370340,2,2.211324e+11,3599022232,1854.610000,...,1.201422e+08,NaN,4878.260000,-62.26963,2021-11-10T14:24:19.604Z,0.432979,424998.32700,2015-10-20T00:00:00.000Z,"{'times': 82.84153065394638, 'currency': 'btc'...",2023-08-15T05:41:25.203Z
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.998728,83304783632,3,8.330478e+10,14202690715,1.001000,...,8.341087e+10,NaN,1.320000,-24.51578,2018-07-24T00:00:00.000Z,0.572521,74.44394,2015-03-02T00:00:00.000Z,None,2023-08-15T05:40:00.817Z
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,239.320000,36820942009,4,4.786411e+10,294399063,241.000000,...,1.538562e+08,2.000000e+08,686.310000,-65.12918,2021-05-10T07:24:17.097Z,0.039818,600941.16904,2017-10-19T00:00:00.000Z,None,2023-08-15T05:41:25.106Z
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.628844,33194653166,5,6.288381e+10,933540158,0.635508,...,9.998852e+10,1.000000e+11,3.400000,-81.49686,2018-01-07T00:00:00.000Z,0.002686,23309.21721,2014-05-22T00:00:00.000Z,None,2023-08-15T05:41:25.648Z
5,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,0.999881,25999359685,6,2.599249e+10,2667299120,1.001000,...,2.599560e+10,NaN,1.170000,-14.73737,2019-05-08T00:40:28.300Z,0.877647,13.92741,2023-03-11T08:02:13.981Z,None,2023-08-15T05:41:26.112Z
6,staked-ether,steth,Lido Staked Ether,https://assets.coingecko.com/coins/images/1344...,1840.040000,14922617884,7,1.491892e+10,3980507,1853.330000,...,8.109614e+06,8.107607e+06,4829.570000,-61.90370,2021-11-10T14:40:47.256Z,482.900000,281.01098,2020-12-22T04:08:21.854Z,None,2023-08-15T05:41:24.371Z
7,dogecoin,doge,Dogecoin,https://assets.coingecko.com/coins/images/5/la...,0.074419,10464413048,8,1.046433e+10,407964573,0.075371,...,1.405947e+11,NaN,0.731578,-89.82582,2021-05-08T05:08:23.458Z,0.000087,85548.83677,2015-05-06T00:00:00.000Z,None,2023-08-15T05:41:22.394Z
8,solana,sol,Solana,https://assets.coingecko.com/coins/images/4128...,24.960000,10135297733,9,1.384980e+10,403404575,25.240000,...,5.549334e+08,NaN,259.960000,-90.40050,2021-11-06T21:54:35.825Z,0.500801,4882.97123,2020-05-11T19:35:23.449Z,None,2023-08-15T05:41:24.947Z
9,cardano,ada,Cardano,https://assets.coingecko.com/coins/images/975/...,0.289123,10133397088,10,1.301192e+10,118306622,0.292078,...,4.500000e+10,4.500000e+10,3.090000,-90.63291,2021-09-02T06:00:10.474Z,0.019253,1401.88277,2020-03-13T02:22:55.044Z,None,2023-08-15T05:41:21.499Z


Voy a definir una función para obtener la *capitalización promedio* en el último año

In [6]:
def obtener_promedio_market_cap(coin_id, start_date, end_date):
    data_historica = cg.get_coin_market_chart_range_by_id(
        id=coin_id,
        vs_currency="usd",
        from_timestamp=start_date.timestamp(),
        to_timestamp=end_date.timestamp()
    )
    
    # Suma de las capitalizaciones de mercado diarias
    total_market_cap = sum(data[1] for data in data_historica['market_caps'])
    
    # Capitalización promedio
    market_cap_promedio = total_market_cap / len(data_historica['market_caps'])
    
    return market_cap_promedio
    
# Fecha de inicio y fin para el último año
end_date = datetime.now()
start_date = end_date - pd.DateOffset(years=1)

# Capitalización promedio en el último año para las criptomonedas
market_cap_promedio = df_coins_markets.apply(lambda row: 
    obtener_promedio_market_cap(row['id'], start_date, end_date), axis=1)

# Aquí se está agregando la capitalización promedio del último año a df_coins_markets
df_coins_markets['promedio_market_cap_ultimo_año'] = market_cap_promedio

# Los números se mostraran con dos decimales
pd.options.display.float_format = '{:.2f}'.format

Si ordenamos por _`promedio_market_cap_ultimo_año`_ podemos ver que las 10 criptomondas con mayor capitalización del momento, han mantenido una posición sólida promedio entre los primeros puestos del ranking de capitalización de mercado durante el último año. Por esta razón, elegiremos estas 10 criptomonedas.

In [7]:
df_coins_markets.head(12).sort_values(by='promedio_market_cap_ultimo_año', ascending=False)

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated,promedio_market_cap_ultimo_año
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,29346.00,571046620112,1,616309030303.00,9516498849,29653.00,...,21000000.00,69045.00,-57.49,2021-11-10T14:24:11.849Z,67.81,43180.46,2013-07-06T00:00:00.000Z,None,2023-08-15T05:41:23.397Z,453558268323.96
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,1840.46,221132370340,2,221132370340.00,3599022232,1854.61,...,NaN,4878.26,-62.27,2021-11-10T14:24:19.604Z,0.43,424998.33,2015-10-20T00:00:00.000Z,"{'times': 82.84153065394638, 'currency': 'btc'...",2023-08-15T05:41:25.203Z,194300817706.21
2,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,1.00,83304783632,3,83304783632.00,14202690715,1.00,...,NaN,1.32,-24.52,2018-07-24T00:00:00.000Z,0.57,74.44,2015-03-02T00:00:00.000Z,None,2023-08-15T05:40:00.817Z,73857212643.01
3,binancecoin,bnb,BNB,https://assets.coingecko.com/coins/images/825/...,239.32,36820942009,4,47864114641.00,294399063,241.00,...,200000000.00,686.31,-65.13,2021-05-10T07:24:17.097Z,0.04,600941.17,2017-10-19T00:00:00.000Z,None,2023-08-15T05:41:25.106Z,45312730681.65
5,usd-coin,usdc,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.00,25999359685,6,25992487220.00,2667299120,1.00,...,NaN,1.17,-14.74,2019-05-08T00:40:28.300Z,0.88,13.93,2023-03-11T08:02:13.981Z,None,2023-08-15T05:41:26.112Z,38673175065.93
4,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,0.63,33194653166,5,62883805276.00,933540158,0.64,...,100000000000.00,3.40,-81.50,2018-01-07T00:00:00.000Z,0.00,23309.22,2014-05-22T00:00:00.000Z,None,2023-08-15T05:41:25.648Z,23055538738.20
9,cardano,ada,Cardano,https://assets.coingecko.com/coins/images/975/...,0.29,10133397088,10,13011916049.00,118306622,0.29,...,45000000000.00,3.09,-90.63,2021-09-02T06:00:10.474Z,0.02,1401.88,2020-03-13T02:22:55.044Z,None,2023-08-15T05:41:21.499Z,12529877370.28
7,dogecoin,doge,Dogecoin,https://assets.coingecko.com/coins/images/5/la...,0.07,10464413048,8,10464327454.00,407964573,0.08,...,NaN,0.73,-89.83,2021-05-08T05:08:23.458Z,0.00,85548.84,2015-05-06T00:00:00.000Z,None,2023-08-15T05:41:22.394Z,10546479713.51
6,staked-ether,steth,Lido Staked Ether,https://assets.coingecko.com/coins/images/1344...,1840.04,14922617884,7,14918924329.00,3980507,1853.33,...,8107606.83,4829.57,-61.90,2021-11-10T14:40:47.256Z,482.90,281.01,2020-12-22T04:08:21.854Z,None,2023-08-15T05:41:24.371Z,9293168051.20
8,solana,sol,Solana,https://assets.coingecko.com/coins/images/4128...,24.96,10135297733,9,13849802771.00,403404575,25.24,...,NaN,259.96,-90.40,2021-11-06T21:54:35.825Z,0.50,4882.97,2020-05-11T19:35:23.449Z,None,2023-08-15T05:41:24.947Z,8544441159.90


Las 10 criptomonedas seleccionadas son las siguientes:

In [8]:
df_10_criptomonedas=df_coins_markets[["id", "symbol", "name"]].head(10)
df_10_criptomonedas

,id,symbol,name
0,bitcoin,btc,Bitcoin
1,ethereum,eth,Ethereum
2,tether,usdt,Tether
3,binancecoin,bnb,BNB
4,ripple,xrp,XRP
5,usd-coin,usdc,USD Coin
6,staked-ether,steth,Lido Staked Ether
7,dogecoin,doge,Dogecoin
8,solana,sol,Solana
9,cardano,ada,Cardano


Los parámetros que elegí para analizar las 10 criptomonedas, son los siguientes:
_`precio`_, _`market_cap`_, _`volumen total`_, _`circulating_supply `_

Definamos cada uno de ellos:
- **precio**: Valor monetario de una criptomoneda en un momento dado.
- **market_cap (capitalización de mercado)**: Valor total de la criptomoneda en el mercado.
- **volumen total**: Cantidad total de una criptomoneda que se ha comprado y vendido en un período dado, generalmente en un día.

*Lido Staked Ether* registra precios recién desde el 22 de diciembre del 2020, entonces para no tenes valores nulos, la data que vamos a extraer será a partir de esa fecha

Primero extraeremos los **precios**

In [9]:
def obtener_precios_diarios_cripto(simbolos_cripto, fecha_inicio, fecha_fin):
    base_url = "https://api.coingecko.com/api/v3"
    datos_historicos = []

    for simbolo in simbolos_cripto:
        url = f"{base_url}/coins/{simbolo}/market_chart/range"
        parametros = {
            "vs_currency": "usd",
            "from": int(fecha_inicio.timestamp()),
            "to": int(fecha_fin.timestamp())
        }

        respuesta = requests.get(url, params=parametros)
        datos = respuesta.json()

        if 'prices' in datos:
            precios = datos['prices']
            df = pd.DataFrame(precios, columns=["timestamp", simbolo])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            datos_historicos.append(df)

    datos_combinados = pd.concat(datos_historicos, axis=1)
    return datos_combinados

simbolos_cripto = ['bitcoin', 'ethereum', 'tether', 'binancecoin', 'ripple', 'usd-coin', 'staked-ether', 'dogecoin', 'cardano', 'solana']
fecha_inicio = datetime(2020, 12, 21)
fecha_fin = datetime.now()

precios_diarios_historicos = obtener_precios_diarios_cripto(simbolos_cripto, fecha_inicio, fecha_fin)


In [10]:
precios_diarios_historicos.tail(10)

,bitcoin,ethereum,tether,binancecoin,ripple,usd-coin,staked-ether,dogecoin,cardano,solana
timestamp,,,,,,,,,,
2023-08-06,29046.80,1835.07,1.00,242.88,0.63,1.00,1835.10,0.08,0.29,22.63
2023-08-07,29044.20,1826.71,1.00,242.57,0.62,1.00,1826.24,0.07,0.29,23.16
2023-08-08,29178.15,1826.93,1.00,241.74,0.62,1.00,1821.72,0.07,0.29,23.05
2023-08-09,29779.56,1856.84,1.00,245.18,0.64,1.00,1856.80,0.08,0.30,24.20
2023-08-10,29585.49,1855.89,1.00,244.01,0.64,1.00,1855.36,0.08,0.30,24.38
2023-08-11,29423.82,1850.88,1.00,241.23,0.63,1.00,1849.45,0.08,0.30,24.62
2023-08-12,29396.85,1847.54,1.00,239.60,0.63,1.00,1846.30,0.08,0.29,24.58
2023-08-13,29412.14,1848.51,1.00,240.09,0.63,1.00,1846.71,0.08,0.29,24.89
2023-08-14,29284.97,1839.51,1.00,240.03,0.63,1.00,1837.91,0.07,0.29,24.22


Ahora extraeremos la **capitalización de mercado histórica**

In [11]:
def obtener_market_cap_diario(simbolos_cripto, fecha_inicio, fecha_fin):
    base_url = "https://api.coingecko.com/api/v3"
    datos_historicos = []

    for simbolo in simbolos_cripto:
        url = f"{base_url}/coins/{simbolo}/market_chart/range"
        parametros = {
            "vs_currency": "usd",
            "from": int(fecha_inicio.timestamp()),
            "to": int(fecha_fin.timestamp())
        }

        respuesta = requests.get(url, params=parametros)
        datos = respuesta.json()

        if 'market_caps' in datos:
            market_caps = datos['market_caps']
            df = pd.DataFrame(market_caps, columns=["timestamp", simbolo])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            datos_historicos.append(df)

    datos_combinados = pd.concat(datos_historicos, axis=1)
    return datos_combinados

market_cap_diario = obtener_market_cap_diario(simbolos_cripto, fecha_inicio, fecha_fin)


In [12]:
market_cap_diario.tail()

,bitcoin,ethereum,tether,binancecoin,ripple,usd-coin,staked-ether,dogecoin,cardano,solana
timestamp,,,,,,,,,,
2023-08-11,572414205818.41,222411268370.90,83445632345.53,37112194985.22,33426764509.19,26152601233.99,14891867268.09,10641945541.16,10375263805.25,9982518848.60
2023-08-12,571976559045.83,222019374228.76,83313809612.90,36864812015.20,33340650585.53,26116498500.48,14861851192.26,10636295250.11,10278658176.64,9969763672.69
2023-08-13,572257798803.91,222069758272.73,83355420569.71,36943552949.56,33093472877.82,26161944529.39,14933314406.10,10774454527.40,10217585439.95,10095650763.65
2023-08-14,569687130446.78,220948326523.33,83303287526.81,36913820931.58,33020932779.51,26180583371.68,14877132696.77,10503954202.79,10135047904.83,9839554742.69
2023-08-15,572242735188.24,221564238004.31,83323482665.83,36970232010.68,33526777989.72,26004359971.61,14932196393.36,10515116058.05,10181963472.17,10211324982.84


Ahora extraeremos el **volumen total**

In [16]:
def obtener_volumen_total_diario(simbolos_cripto, fecha_inicio, fecha_fin):
    base_url = "https://api.coingecko.com/api/v3"
    datos_historicos = []

    for simbolo in simbolos_cripto:
        url = f"{base_url}/coins/{simbolo}/market_chart/range"
        parametros = {
            "vs_currency": "usd",
            "from": int(fecha_inicio.timestamp()),
            "to": int(fecha_fin.timestamp())
        }

        respuesta = requests.get(url, params=parametros)
        datos = respuesta.json()

        if 'total_volumes' in datos:
            total_volumes = datos['total_volumes']
            df = pd.DataFrame(total_volumes, columns=["timestamp", simbolo])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            datos_historicos.append(df)

    datos_combinados = pd.concat(datos_historicos, axis=1)
    return datos_combinados

volumen_total_diario = obtener_volumen_total_diario(simbolos_cripto, fecha_inicio, fecha_fin)


In [17]:
volumen_total_diario.tail()

,bitcoin,ethereum,tether,binancecoin,ripple,usd-coin,staked-ether,dogecoin,cardano,solana
timestamp,,,,,,,,,,
2023-08-11,8106253519.03,3733537744.90,10890096033.92,339047442.16,1063357974.34,2203564212.02,22500456.48,399366004.77,179477802.60,352124557.45
2023-08-12,4651950357.64,3694816570.96,8184751224.06,363763402.78,924970915.64,1575234195.34,22880008.91,263740989.01,124610584.57,337807060.84
2023-08-13,3994297895.29,2215986526.70,9189020281.95,195205362.18,502921367.18,1242418455.69,2289124.38,385515900.69,105607968.60,277306903.12
2023-08-14,4755220448.63,3642424563.74,9254138040.71,230763796.03,620939906.98,1681826723.97,3616811.89,325530830.83,125106297.16,278182091.44
2023-08-15,12408355477.11,4495324729.87,12792365654.79,323212804.63,967305278.57,2461885540.00,5027849.07,433958600.07,129874470.40,388025853.45


Hasta ahora tengo 3 Data Frames, pero también necesito un DF para cada criptomoneda, para luego elegir con cuál de los DataFrames es más óptimo trabajar
Para conseguir un DF para cada criptomoneda con los tres parámetros que elegimos, definimos la siguiente función

In [18]:
def obtener_datos_cripto(criptomoneda, fecha_inicio, fecha_fin):
    base_url = "https://api.coingecko.com/api/v3"
    url = f"{base_url}/coins/{criptomoneda}/market_chart/range"
    parametros = {
        "vs_currency": "usd",
        "from": int(fecha_inicio.timestamp()),
        "to": int(fecha_fin.timestamp())
    }

    respuesta = requests.get(url, params=parametros)
    datos = respuesta.json()

    df = pd.DataFrame(datos['prices'], columns=['timestamp', 'precio'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    df['market_cap'] = [entry[1] for entry in datos['market_caps']]
    df['total_volumen'] = [entry[1] for entry in datos['total_volumes']]
    
    return df[['precio', 'market_cap', 'total_volumen']]

fecha_inicio = datetime(2020, 12, 21)
fecha_fin = datetime.now()

In [20]:
criptomoneda_elegida = 'bitcoin'
bitcoin_df = obtener_datos_cripto(criptomoneda_elegida, fecha_inicio, fecha_fin)
bitcoin_df.tail()

,precio,market_cap,total_volumen
timestamp,,,
2023-08-11,29423.82,572414205818.41,8106253519.03
2023-08-12,29396.85,571976559045.83,4651950357.64
2023-08-13,29412.14,572257798803.91,3994297895.29
2023-08-14,29284.97,569687130446.78,4755220448.63
2023-08-15,29400.59,572242735188.24,12408355477.11


In [21]:
criptomoneda_elegida = 'ethereum'
ethereum_df = obtener_datos_cripto(criptomoneda_elegida, fecha_inicio, fecha_fin)
ethereum_df.tail()

,precio,market_cap,total_volumen
timestamp,,,
2023-08-11,1850.88,222411268370.90,3733537744.90
2023-08-12,1847.54,222019374228.76,3694816570.96
2023-08-13,1848.51,222069758272.73,2215986526.70
2023-08-14,1839.51,220948326523.33,3642424563.74
2023-08-15,1843.51,221564238004.31,4495324729.87


In [22]:
criptomoneda_elegida = 'tether'
tether_df = obtener_datos_cripto(criptomoneda_elegida, fecha_inicio, fecha_fin)
tether_df.tail()

,precio,market_cap,total_volumen
timestamp,,,
2023-08-11,1.00,83445632345.53,10890096033.92
2023-08-12,1.00,83313809612.90,8184751224.06
2023-08-13,1.00,83355420569.71,9189020281.95
2023-08-14,1.00,83303287526.81,9254138040.71
2023-08-15,1.00,83323482665.83,12792365654.79


In [23]:
criptomoneda_elegida = 'binancecoin'
binancecoin_df = obtener_datos_cripto(criptomoneda_elegida, fecha_inicio, fecha_fin)
binancecoin_df.tail()

,precio,market_cap,total_volumen
timestamp,,,
2023-08-11,241.23,37112194985.22,339047442.16
2023-08-12,239.60,36864812015.20,363763402.78
2023-08-13,240.09,36943552949.56,195205362.18
2023-08-14,240.03,36913820931.58,230763796.03
2023-08-15,240.36,36970232010.68,323212804.63


In [24]:
criptomoneda_elegida = 'ripple'
ripple_df = obtener_datos_cripto(criptomoneda_elegida, fecha_inicio, fecha_fin)
ripple_df.tail()

,precio,market_cap,total_volumen
timestamp,,,
2023-08-11,0.63,33426764509.19,1063357974.34
2023-08-12,0.63,33340650585.53,924970915.64
2023-08-13,0.63,33093472877.82,502921367.18
2023-08-14,0.63,33020932779.51,620939906.98
2023-08-15,0.63,33526777989.72,967305278.57


In [25]:
criptomoneda_elegida = 'usd-coin'
usd_coin_df = obtener_datos_cripto(criptomoneda_elegida, fecha_inicio, fecha_fin)
usd_coin_df.tail()

,precio,market_cap,total_volumen
timestamp,,,
2023-08-11,1.00,26152601233.99,2203564212.02
2023-08-12,1.00,26116498500.48,1575234195.34
2023-08-13,1.00,26161944529.39,1242418455.69
2023-08-14,1.00,26180583371.68,1681826723.97
2023-08-15,1.00,26004359971.61,2461885540.00


In [26]:
criptomoneda_elegida = 'staked-ether'
staked_ether_df = obtener_datos_cripto(criptomoneda_elegida, fecha_inicio, fecha_fin)
staked_ether_df.tail()

,precio,market_cap,total_volumen
timestamp,,,
2023-08-11,1849.45,14891867268.09,22500456.48
2023-08-12,1846.30,14861851192.26,22880008.91
2023-08-13,1846.71,14933314406.10,2289124.38
2023-08-14,1837.91,14877132696.77,3616811.89
2023-08-15,1842.55,14932196393.36,5027849.07


In [32]:
criptomoneda_elegida = 'dogecoin'
dogecoin_df = obtener_datos_cripto(criptomoneda_elegida, fecha_inicio, fecha_fin)
dogecoin_df.tail()

,precio,market_cap,total_volumen
timestamp,,,
2023-08-11,0.08,10641945541.16,399366004.77
2023-08-12,0.08,10636295250.11,263740989.01
2023-08-13,0.08,10774454527.40,385515900.69
2023-08-14,0.07,10503954202.79,325530830.83
2023-08-15,0.07,10515116058.05,433958600.07


In [28]:
criptomoneda_elegida = 'cardano'
cardano_df = obtener_datos_cripto(criptomoneda_elegida, fecha_inicio, fecha_fin)
cardano_df.tail()

,precio,market_cap,total_volumen
timestamp,,,
2023-08-11,0.30,10375263805.25,179477802.60
2023-08-12,0.29,10278658176.64,124610584.57
2023-08-13,0.29,10217585439.95,105607968.60
2023-08-14,0.29,10135047904.83,125106297.16
2023-08-15,0.29,10181963472.17,129874470.40


In [29]:
criptomoneda_elegida = 'solana'
solana_df = obtener_datos_cripto(criptomoneda_elegida, fecha_inicio, fecha_fin)
solana_df.tail()

,precio,market_cap,total_volumen
timestamp,,,
2023-08-11,24.62,9982518848.60,352124557.45
2023-08-12,24.58,9969763672.69,337807060.84
2023-08-13,24.89,10095650763.65,277306903.12
2023-08-14,24.22,9839554742.69,278182091.44
2023-08-15,25.15,10211324982.84,388025853.45


Como CoinGecko está en constante actualización, es preferible guardar los datos extraídos hasta el día de hoy **(15/08/2023)** <br>
Guardé mi dataset en formato CVS para trabajar el EDA, luego el dashboard, para finalmente hacer el análisis

In [30]:
df_10_criptomonedas.to_csv("Data/top_10_criptomonedas.csv")
precios_diarios_historicos.to_csv("Data/precios_diarios_historicos.csv")
market_cap_diario.to_csv("Data/market_cap_diario.csv")
volumen_total_diario.to_csv("Data/volumen_total_diario.csv")
bitcoin_df.to_csv("Data/bitcoin.csv")
ethereum_df.to_csv("Data/ethereum.csv")
tether_df.to_csv("Data/tether.csv")
binancecoin_df.to_csv("Data/binancecoin.csv")
ripple_df.to_csv("Data/ripple.csv")
usd_coin_df.to_csv("Data/USDcoin.csv")
staked_ether_df.to_csv("Data/StakedEther.csv")
dogecoin_df.to_csv("Data/dogecoin.csv")
cardano_df.to_csv("Data/cardano.csv")
solana_df.to_csv("Data/solana.csv")